# Инфраструктура для моделей машинного обучения

Использование библиотек PySpark SQL и PySpark ML для предобработки данных и обучения моделей.

# Что входит в работу

1. Инициализация спарк-сессии.
2. Загрузка данных.
3. Ознакомление с данными.
4. Преобразование типов столбцов.
5. Очистка данных.
6. Feature-инжиниринг.
7. Векторизация фичей.
8. Создание и обучение модели.
9. Выбор лучшей модели.

# Задача

Используя данные о клиентах телекоммуникационной компании, обучите модель, предсказывающую их отток.

Описание данных, с которыми вы будете работать:

* **CustomerID**: ID клиента.
* **Gender**: пол клиента.
* **SeniorCitizen**: пенсионер ли клиент (1 — да, 0 — нет).
* **Partner**: есть у клиента партнёр (жена, муж) или нет (Yes/No).
* **Dependents**: есть ли у клиента инждивенцы, например дети (Yes/No).
* **Tenure**: как много месяцев клиент оставался в компании.
* **PhoneService**: подключена ли у клиента телефонная служба (Yes/No).
* **MultipleLines**: подключено ли несколько телефонных линий (Yes, No, No phone service).
* **InternetService**: интернет-провайдер клиента (DSL, Fiber optic, No).
* **OnlineSecurity**: подключена ли у клиента услуга онлайн-безопасности (Yes, No, No internet service)
* **OnlineBackup**: подключена ли услуга резервного копирования онлайн (Yes, No, No internet service).
* **DeviceProtection**: подключена ли услуга защиты устройства (Yes, No, No internet service)
* **TechSupport**: есть ли у клиента техническая поддержка (Yes, No, No internet service).
* **StreamingTV**: подключена ли услуга потокового телевидения (Yes, No, No internet service).
* **StreamingMovies**: подключена ли услуга стримингового воспроизведения фильмов (Yes, No, No internet service).
* **Contract**: тип контракта клиента (Month-to-month, One year, Two year).
* **PaperlessBilling**: есть ли безбумажный счёт.
* **PaymentMethod**: способ оплаты услуг (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)).
* **MonthlyCharges**: сумма, которая списывается ежемесячно.
* **TotalCharges**: сумма, списанная за всё время.
* **Churn**: ушёл ли клиент (Yes/No). Это целевая переменная, которую нужно предсказать.


# 1. Инициализация спарк-сессии

Инициализируйте спарк-сессию.

Эта ячейка нужна для того, чтобы заргузить необходимые библиотеки и настроить окружение Google Colab для работы со Spark.

In [1]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

# 2. Загрузка данных
Загрузка данных, сохранение их в переменную типа sparkDataframe, используя метод read.csv

In [3]:
# Загрузка файла в Google Colab
from google.colab import drive
drive.mount("/content/drive")

sparkDataframe = spark.read.option("header",True).option("delimiter",";").csv("/content/drive/MyDrive/Colab Notebooks/WA_Fn-UseC_-Telco-Customer-Churn.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Ознакомление с данными
1. Вывод на экран первых несколько строк датафрейма.


In [4]:
sparkDataframe.show()

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                     

2. Вывод общего количества строк датафрейма.

In [5]:
# Вывод общего количества строк
print(f"общее количество строк датафрейма: {sparkDataframe.count()}")

общее количество строк датафрейма: 7043


3. Вывод структуры (схемы) датафрейма.

In [6]:
sparkDataframe.printSchema()

root
 |-- customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn: string (nullable = true)



# 4. Преобразование типов столбцов
Преобразование типа столбцов у числовых признаков (Int — если признак целочисленный, Double — если признак не целочисленный). Сохранение преобразованного датафрейма в новую переменную.

In [7]:
from pyspark.sql.functions import expr, col, count, split

column = sparkDataframe.columns[0]
column

# разделение исходных данных на несколько колонок
sparkDataframe1 = sparkDataframe.withColumn('gender', split(sparkDataframe[column], ',').getItem(1)) \
                  .withColumn('SeniorCitizen', split(sparkDataframe[column], ',').getItem(2)) \
                  .withColumn('Partner', split(sparkDataframe[column], ',').getItem(3)) \
                  .withColumn('Dependents', split(sparkDataframe[column], ',').getItem(4)) \
                  .withColumn('tenure', split(sparkDataframe[column], ',').getItem(5).cast("Int")) \
                  .withColumn('PhoneService', split(sparkDataframe[column], ',').getItem(6)) \
                  .withColumn('MultipleLines', split(sparkDataframe[column], ',').getItem(7)) \
                  .withColumn('InternetService', split(sparkDataframe[column], ',').getItem(8)) \
                  .withColumn('OnlineSecurity', split(sparkDataframe[column], ',').getItem(9)) \
                  .withColumn('OnlineBackup', split(sparkDataframe[column], ',').getItem(10)) \
                  .withColumn('DeviceProtection', split(sparkDataframe[column], ',').getItem(11)) \
                  .withColumn('TechSupport', split(sparkDataframe[column], ',').getItem(12)) \
                  .withColumn('StreamingTV', split(sparkDataframe[column], ',').getItem(13)) \
                  .withColumn('StreamingMovies', split(sparkDataframe[column], ',').getItem(14)) \
                  .withColumn('Contract', split(sparkDataframe[column], ',').getItem(15)) \
                  .withColumn('PaperlessBilling', split(sparkDataframe[column], ',').getItem(16)) \
                  .withColumn('PaymentMethod', split(sparkDataframe[column], ',').getItem(17)) \
                  .withColumn('MonthlyCharges', split(sparkDataframe[column], ',').getItem(18).cast("Double")) \
                  .withColumn('TotalCharges', split(sparkDataframe[column], ',').getItem(19).cast("Double")) \
                  .withColumn('Churn', split(sparkDataframe[column], ',').getItem(20)).drop(column)

sparkDataframe1.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|

# 5. Очистка данных
Провека, есть ли в какой-либо колонке Null-значения. Для этого можно использовать your_dataframe.filter(col("colname")).isNull()).

Вывод на экран несколько строк с Null-значениями в одной из колонок.

Сохранение очищенных от строк с Null-значениями датафрейма в новую переменную. Для фильтрации этих значений можно использовать метод .isNotNull().

Колонок в датафрейме много, проверять каждую неудобно и долго. Упроситить эту работу, если использовать, например, перебор с циклом for.

In [8]:
columns = sparkDataframe1.columns
columns

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [9]:
# количество пропусков в каждой колонке

for col in columns:
  print(col, sparkDataframe1.filter(sparkDataframe1[col].isNull()).count())

gender 0
SeniorCitizen 0
Partner 0
Dependents 0
tenure 0
PhoneService 0
MultipleLines 0
InternetService 0
OnlineSecurity 0
OnlineBackup 0
DeviceProtection 0
TechSupport 0
StreamingTV 0
StreamingMovies 0
Contract 0
PaperlessBilling 0
PaymentMethod 0
MonthlyCharges 0
TotalCharges 11
Churn 0


In [10]:
sparkDataframe1.filter(sparkDataframe1['TotalCharges'].isNull()).show()   # вывод строк с нулевыми значениями в колонке TotalCharges

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|       Yes|     0|          No|No phone service|            DSL|                Ye

In [11]:
sparkDataframe2 = sparkDataframe1.filter(sparkDataframe1['TotalCharges'].isNotNull()) # запись нового дф без строк с пропусками

In [12]:
sparkDataframe2.filter(sparkDataframe2['TotalCharges'].isNull()).show()   # проверка, что строк с нулевыми значениями в колонке TotalCharges не осталось

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+---

# 6. Feature-инжиниринг
Добавление в датафрейм одну или несколько новых фичей. Удаление колонок, которые, как мне кажется, нужно убрать из фичей.

In [13]:
from pyspark.sql.functions import when

sparkDataframe2.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|

In [14]:
# Удаляю колонку CustomerID, так как она не несет никакой полезной информации для модели, SeniorCitizen, так как клиентами компании являются лица разных возрастных категорий
sparkDataframe2 = sparkDataframe2.drop('CustomerID', 'SeniorCitizen')

# Создаю новую фичу, которая будет показывать, сколько лет клиент пользуется услугами компании.
sparkDataframe2 = sparkDataframe2.withColumn('tenure_years', sparkDataframe2['tenure'] / 12)

# Удаляю колонку tenure, так как она теперь дублируется в колонке tenure_years.
sparkDataframe2 = sparkDataframe2.drop('tenure')

sparkDataframe2.show()

+------+-------+----------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+-------------------+
|gender|Partner|Dependents|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|       tenure_years|
+------+-------+----------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+-------------------+
|Female|    Yes|        No|          No|No phone service|            DSL|                 No|    

In [15]:
# Удаляю колонку CustomerID, так как она не несет никакой полезной информации для модели.
# CustomerID - это просто идентификатор клиента, который не имеет отношения к его поведению или характеристикам.
# SeniorCitizen, так как клиентами компании являются лица разных возрастных категорий

# Создадаю новую фичу, которая будет показывать, сколько лет клиент пользуется услугами компании.
# Эта фича может быть более полезной для модели, чем исходная фича tenure, которая показывает количество месяцев, так как она позволяет оценить, насколько давно клиент пользуется услугами компании.

# Удаляю колонку tenure, так как она теперь дублируется в колонке tenure_years.
# Колонка tenure теперь не нужна, так как вся необходимая информация о продолжительности пользования услугами компании содержится в колонке tenure_years.


# 7. Векторизация фичей
Подготовка данных к обучению:

1. Преобразую текстовые колонки в числа, используя StringIndexer.
Удаление столбцов со старыми (непреобразованными) признаками. Вывод на экран структуры получившегося датафрейма. Не забываю о столбце Churn. Хоть он и выступает в задаче как таргет, он имеет текстовый тип, поэтому тоже должен быть закодирован числовыми значениями.

Чтобы использовать StringIndexer для всех категориальных признаков сразу, а не для каждого отдельно, можно применить сущность pipeline.

**Пример кода:**

##### #Задаём список текстовых колонок:
text_columns = ["text_col_1", "text_col_2", "text_col_3"]

##### #Задаём список StringIndexer'ов — сущностей, каждая из которых будет кодировать одну текстовую колонку числами. Имена преобразованных колонок будут заканчиваться на _index:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(<ваш датасет>) for column in text_columns]

##### #Создаём Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

##### #Скармливаем нашему pipeline датафрейм, удаляя старые колонки:
new_dataframe = pipeline.fit(<ваш датасет>).transform(<ваш датасет>).drop(*text_columns)


In [16]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#список колонок с текстовым типом
text_cols = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "Churn"]


# Преобразую текстовые колонки в числовые значения
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in text_cols]

#Создаю Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

#Преобразую pipeline датафрейм, удаляя старые колонки:
sparkDataframe3 = pipeline.fit(sparkDataframe2).transform(sparkDataframe2).drop(*text_cols)

sparkDataframe3.printSchema()

root
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- tenure_years: double (nullable = true)
 |-- gender_index: double (nullable = false)
 |-- Partner_index: double (nullable = false)
 |-- Dependents_index: double (nullable = false)
 |-- PhoneService_index: double (nullable = false)
 |-- MultipleLines_index: double (nullable = false)
 |-- InternetService_index: double (nullable = false)
 |-- OnlineSecurity_index: double (nullable = false)
 |-- OnlineBackup_index: double (nullable = false)
 |-- DeviceProtection_index: double (nullable = false)
 |-- TechSupport_index: double (nullable = false)
 |-- StreamingTV_index: double (nullable = false)
 |-- StreamingMovies_index: double (nullable = false)
 |-- Contract_index: double (nullable = false)
 |-- PaperlessBilling_index: double (nullable = false)
 |-- PaymentMethod_index: double (nullable = false)
 |-- Churn_index: double (nullable = false)



2. Векторизуйю категориальные признаки, используя OneHotEncoder.
Удаляю столбцы со старыми (непреобразованными) признаками.
Вывод на экран структуры получившегося после преобразований датафрейма.


In [17]:
from pyspark.ml.feature import OneHotEncoder

#список категориальных колонок
features_inp  = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"]

features_inp = [s+'_index' for s in features_inp]

features_out = [s+'_vec' for s in features_inp]

encoder = OneHotEncoder(inputCols=features_inp, outputCols=features_out)

In [18]:
encoded = encoder.fit(sparkDataframe3)
sparkDataframe3 = encoded.transform(sparkDataframe3).drop("PhoneService", "MultipleLines", "InternetService", \
               "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
               "StreamingMovies", "Contract", "PaymentMethod")


sparkDataframe3.printSchema()

root
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- tenure_years: double (nullable = true)
 |-- gender_index: double (nullable = false)
 |-- Partner_index: double (nullable = false)
 |-- Dependents_index: double (nullable = false)
 |-- PhoneService_index: double (nullable = false)
 |-- MultipleLines_index: double (nullable = false)
 |-- InternetService_index: double (nullable = false)
 |-- OnlineSecurity_index: double (nullable = false)
 |-- OnlineBackup_index: double (nullable = false)
 |-- DeviceProtection_index: double (nullable = false)
 |-- TechSupport_index: double (nullable = false)
 |-- StreamingTV_index: double (nullable = false)
 |-- StreamingMovies_index: double (nullable = false)
 |-- Contract_index: double (nullable = false)
 |-- PaperlessBilling_index: double (nullable = false)
 |-- PaymentMethod_index: double (nullable = false)
 |-- Churn_index: double (nullable = false)
 |-- gender_index_vec: vector (nullable = true)
 |-- 

3. Объединение колонки фичей в один вектор, используя VectorAssembler.
Удаление столбцов со старыми (непреобразованными) признаками.
Вывод на экран первых несколько строк и структуры получившегося датафрейма.

In [19]:
from pyspark.ml.feature import VectorAssembler

features = list(sparkDataframe3.drop("Churn_index").columns) #Список колонок фичей

vectorizer = VectorAssembler(inputCols = features, outputCol = "features_vec")
#Создаю переменную vectorizer — объект класса VectorAssembler, цель которого — превратить колонки с фичами в векторы

new_df = vectorizer.transform(sparkDataframe3)
#Трансформирую при помощи vectorizer датафрейм и сохраняю его в переменную telecom_vectorised

new_df.show()

new_df.select("features_vec","Churn_index").show()

+--------------+------------+-------------------+------------+-------------+----------------+------------------+-------------------+---------------------+--------------------+------------------+----------------------+-----------------+-----------------+---------------------+--------------+----------------------+-------------------+-----------+----------------+-----------------+--------------------+----------------------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+------------------+--------------------------+-----------------------+--------------------+
|MonthlyCharges|TotalCharges|       tenure_years|gender_index|Partner_index|Dependents_index|PhoneService_index|MultipleLines_index|InternetService_index|OnlineSecurity_index|OnlineBackup_index|DeviceProtection_index|TechSupport_index|StreamingTV_index|StreamingMovies_index|Contract_index

# 8. Создание и обучение модели

1. Создаю модель — логистическую регрессию (используя LogisticRegression). В качестве параметров класса LogisticRegression укажите колонку фичей (параметр featuresCol), колонку-таргет (параметр labelCol) из датафрейма и имя колонки, в которую будут записываться предсказания (параметр predictionCol).

In [20]:
from pyspark.ml.classification import LogisticRegression

# Создаю модель логистической регрессии
lr = LogisticRegression(featuresCol="features_vec", labelCol="Churn_index", predictionCol="prediction")


2. Разделяю датафрейм на обучающую и тестовую выборку.

In [21]:
# Разделение данных на обучающую и тестовую выборки
train_data, test_data = new_df.randomSplit([0.8, 0.2], seed=1234)

3. Создание объекта — сетки гиперпараметров для каждой модели, используя ParamGridBuilder. В сетку гиперпараметров можно добавить значения параметров regParam и elasticNetParam.

In [22]:
from pyspark.ml.tuning import ParamGridBuilder

# Создание сетки гиперпараметров
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.1, 1.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .build())

4. Создание объекта evaluator, который будет отвечать за метрику качества при обучении. Для этого использую класс BinaryClassificationEvaluator со следующими параметрами: rawPredictionCol — колонка с предсказаниями, labelCol — колонка с таргетом.

По умолчанию BinaryClassificationEvaluator будет рассчитывать areaUnderROC. Это метрика оценки площади под кривой ROC (Receiver Operating Characteristic), которая служит графической интерпретацией производительности модели. Эта метрика качества находится в пределах от 0 до 1. Чем выше метрика, тем более качественные предсказания делает модель.

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Создание объекта evaluator для оценки модели
evaluate = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Churn_index")

5. Создание объекта CrossValidator, в качестве параметров указывю уже созданную модель, сетку гиперпараметров и evaluator.

In [24]:
from pyspark.ml.tuning import CrossValidator

# Создание объекта CrossValidator
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluate,
                          numFolds=5)  # Указываем количество фолдов для кросс-валидации


6. Запуск обучения модели на тренировочной выборке. Сохранение обученной модели в новую переменную.

In [25]:
# Обучение модели с использованием CrossValidator и сохранение обученной модели в переменную
cv_model = crossval.fit(train_data)

# 9. Выбор лучшей модели

1. Выбор лучшей модели, сохранение её в отдельную переменную, отображение её параметров.

Вывод параметров модели в PySpark можно сделать, используя метод extractParamMap().

In [26]:
best_model = cv_model.bestModel

print("Best Model Parameters:")
for param, value in best_model.extractParamMap().items():
    print(f"{param.name}: {value}")

Best Model Parameters:
aggregationDepth: 2
elasticNetParam: 0.5
family: auto
featuresCol: features_vec
fitIntercept: True
labelCol: Churn_index
maxBlockSizeInMB: 0.0
maxIter: 100
predictionCol: prediction
probabilityCol: probability
rawPredictionCol: rawPrediction
regParam: 0.01
standardization: True
threshold: 0.5
tol: 1e-06


2. Запуск лучшей модели в режиме предсказания на тестовой выборке. Сохранение предсказания в отдельную переменную. Вывод первых несколько строк датафрейма с предсказаниями на экран.

Запуск модели в режиме предсказания выполняется при помощи метода .transform(<тестовая выборка>).

In [27]:
prediction = best_model.transform(train_data)
prediction.show()

+--------------+------------+-------------------+------------+-------------+----------------+------------------+-------------------+---------------------+--------------------+------------------+----------------------+-----------------+-----------------+---------------------+--------------+----------------------+-------------------+-----------+----------------+-----------------+--------------------+----------------------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+------------------+--------------------------+-----------------------+--------------------+--------------------+--------------------+----------+
|MonthlyCharges|TotalCharges|       tenure_years|gender_index|Partner_index|Dependents_index|PhoneService_index|MultipleLines_index|InternetService_index|OnlineSecurity_index|OnlineBackup_index|DeviceProtection_index|TechSupport_index|S

3. Получение метрики качества модели. Для этого применяю к объекту evaluator метод .evaluate(<ваш датафрейм с предсказаниями>).

In [30]:
score = evaluate.evaluate(prediction)
print(f"Area Under ROC: {score}")

Area Under ROC: 0.8469131165158976
